# **KawsAI - Un Modelo de IA para Matching de ChambeaYa**

En este notebook se guarda el proceso que se hizo para crear el modelo de IA KawsAI. El objetivo de KawsAI, es hacer el matching con la mejor mype a cada estudiante. Así, el estudiante y la mype pueden conectar para poder trabajar juntos.

### **I. Versión:**
1.0

### **II. Objetivo:**
El objetivo de KawsAI es lograr hacer el mejor match entre estudiantes y un puesto ofrecido por una mype. Este match se va a basar en datos que ChambeaYa va a recopilar de ambos lados a traves de su página web. El objetivo principal es lograr indicar a un estudiante cual es el puesto que ofrece una mype que mas coincide con sus conocimientos, y así aumentar mucho las probabilidades de conexión.

### **III. Descripción de los Datasets**
Para la versión 1.0 vamos a usar datasets simulados con la ayuda de la IA de ChatGPT. Se le pidió a ChatGPT simular 2 datasets en forma de tablas y en formato .csv, que son estudiantes_test.csv y mypes_test.csv. Cada una de las columnas de estos datasets contiene información que se va a solicitar a los usuarios ingresar en nuestra plataforma.

##### IIIa. Descripción de columnas de estudiantes:
- **id_estudiante:** ID único del estudiante (string)  
- **nombre_completo:** Nombre completo del estudiante (string)  
- **correo:** Correo electrónico del estudiante (string)  
- **carrera:** Carrera que cursa el estudiante (string)  
- **ciclo_actual:** Ciclo o semestre actual del estudiante (int)  
- **ubicacion:** Ubicación de residencia del estudiante (string)  
- **areas_interes:** Áreas profesionales de interés del estudiante (string)  
- **habilidades_destacadas:** Habilidades más desarrolladas del estudiante (string)  
- **motivacion_principal:** Motivación principal del estudiante (string)  
- **descripcion_personal:** Descripción libre sobre personalidad y hobbies (string)  
- **horas_semanales_disponibles:** Horas disponibles por semana para trabajar (int)  
- **modalidad_preferida:** Modalidad preferida de trabajo (Presencial, Híbrido, Remoto) (string)  
- **experiencia_relevante:** Experiencia previa relevante (string)  
- **link_portafolio_cv:** Enlace a portafolio o CV del estudiante (string) 

##### IIIb. Descripción de columnas de puestos de mypes:
- **id_puesto:** ID único del puesto ofrecido (string)  
- **titulo_puesto:** Título del puesto ofrecido (string)  
- **descripcion_puesto:** Descripción detallada del puesto (string)  
- **area_del_puesto:** Área o departamento del puesto (string)  
- **modalidad_de_trabajo:** Modalidad del trabajo (Presencial, Híbrido, Remoto) (string)  
- **horas_semanales_requeridas:** Horas semanales requeridas (int)  
- **fecha_inicio:** Fecha estimada de inicio del puesto (date)  
- **duracion_del_puesto:** Duración del puesto en semanas (int)  
- **sueldo_aproximado:** Sueldo aproximado ofrecido (int)

### **IV. Índice**
1. Carga de Librerias

## 1. Carga de Librerias

In [4]:
# Todas las librerias que se deben importar
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd
import numpy as np
import spacy
from spacy.lang.es.stop_words import STOP_WORDS as spacy_stopwords
import requests
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from scipy.sparse import hstack
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors

geolocator = Nominatim(user_agent="chambeaya_app")

In [12]:
!python -m spacy download es_core_news_sm

ERROR: Could not install packages due to an OSError: HTTPSConnectionPool(host='github.com', port=443): Max retries exceeded with url: /explosion/spacy-models/releases/download/es_core_news_sm-3.8.0/es_core_news_sm-3.8.0-py3-none-any.whl (Caused by ConnectTimeoutError(<pip._vendor.urllib3.connection.HTTPSConnection object at 0x77e849357ce0>, 'Connection to github.com timed out. (connect timeout=15)'))



## 2. Carga de datos en DataFrames

In [5]:
# Ahora leemos los csv y los guardamos en dataframes de pandas
estudiantes = pd.read_csv("../datasets/estudiantes_test.csv", index_col=False)
puestos = pd.read_csv("../datasets/puestos_test.csv", index_col=False)

## 3. Preprocesamiento de los datos

### Para los estudiantes:

#### La función de preprocesamiento va a hacer diferentes pasos para diferentes columnas:

**1)** Vamos a juntar todas las columnas textuales en una sola para el modelo y se le van a dar un peso a cada columna, luego se aplicará tf-idf a todas estas columnas:
- carrera (peso de 1)
- habilidades_destacadas (peso de 3)
- areas_interes (peso de 3)
- motivacion_principal (peso de 1)
- descripcion_personal (peso de 2)
- experiencia_relevante (peso de 5)

**2)** Vamos a aplicar one-hot encoding para la columna Modalidad

**3)** Para las columnas numericas vamos a usar StandardScaler, las columnas son:
- ciclo_actual
- horas_semanales_disponibles

**4)** Se van a eliminar las siguientes columnas identificadoras que no aportan nada al modelo:
- id_estudiante
- nombre_completo
- correo
- link_portafolio_cv

In [8]:
class PreprocesadorEstudiantes:
    def __init__(self):
        self.vectorizador_texto = TfidfVectorizer(max_features=500)
        self.escalador = StandardScaler()
        self.columnas_numericas = ['ciclo_actual', 'horas_semanales_disponibles']
        self.columnas_modalidad = []
        self.nlp = spacy.load("es_core_news_sm")

    def _procesar_texto(self, estudiante):
        perfil_textual = (
            ("carrera: " + str(estudiante['carrera']) + " ") * 1 +
            ("habilidades_destacadas: " + str(estudiante['habilidades_destacadas']) + " ") * 3 +
            ("areas_interes: " + str(estudiante['areas_interes']) + " ") * 3 +
            ("descripcion_personal: " + str(estudiante['descripcion_personal']) + " ") * 3 +
            ("experiencia_relevante: " + str(estudiante['experiencia_relevante']) + " ") * 5
        )
        perfil_textual = re.sub(r"[^\w\s]", " ", perfil_textual.lower())
        perfil_textual = re.sub(r"\s+", " ", perfil_textual).strip()

        doc = self.nlp(perfil_textual)
        tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and not token.like_num]
        estudiante['perfil_textual'] = " ".join(tokens)
        return estudiante

    def fit_transform(self, df):
        # Preprocesar textos
        df['perfil_textual'] = df.apply(self._procesar_texto, axis=1)
        X_text = self.vectorizador_texto.fit_transform(df['perfil_textual'])

        # One-hot encoding de modalidad
        df_encoded = pd.get_dummies(df, columns=['modalidad'], drop_first=False)
        self.columnas_modalidad = [col for col in df_encoded.columns if col.startswith('modalidad_')]

        # Escalar numéricas
        df_encoded[self.columnas_numericas] = self.escalador.fit_transform(df_encoded[self.columnas_numericas])

        # Guardamos columna de texto TF-IDF + las demás
        columnas_finales = self.columnas_numericas + self.columnas_modalidad
        X_other = df_encoded[columnas_finales].values
        return hstack([X_text, X_other])

    def transform(self, df_nuevo):
        # Preprocesar texto
        df_nuevo['perfil_textual'] = df_nuevo.apply(self._procesar_texto, axis=1)
        X_text = self.vectorizador_texto.transform(df_nuevo['perfil_textual'])

        # One-hot encoding y alinear columnas
        df_nuevo = pd.get_dummies(df_nuevo, columns=['modalidad'], drop_first=False)
        for col in self.columnas_modalidad:
            if col not in df_nuevo.columns:
                df_nuevo[col] = 0
        df_nuevo = df_nuevo.reindex(columns=self.columnas_numericas + self.columnas_modalidad, fill_value=0)

        # Escalar numéricas
        df_nuevo[self.columnas_numericas] = self.escalador.transform(df_nuevo[self.columnas_numericas])

        return hstack([X_text, df_nuevo.values])

In [9]:
preprocesador = PreprocesadorEstudiantes()
X = preprocesador.fit_transform(estudiantes)

kmeans = KMeans(n_clusters=10, random_state=42)
clusters = kmeans.fit_predict(X.toarray())

estudiantes["cluster"] = clusters

KeyboardInterrupt: 

## 4. Realizamos el 1er filtrado de Mypes de acuerdo a las características del estudiante 

In [32]:
# Con esta función, vamos a filtrar las mypes por las columnas que podemos relacionar directamente con el estudiante, como horas_requeridas
# ubicación y modalidad_preferida

# Para llamar esta función, vamos a realizar este filtrado por cada estudiante que quiera ver que mypes le hacen un mejor match
def filtrar_por_columnas(indice_estudiante, estudiantes, mypes):
    estudiante = estudiantes.iloc[indice_estudiante]
    modalidad = estudiante["modalidad_preferida"]
    horas_disponibles = estudiante["disponibilidad_semanal"]

    print(f"{modalidad} {horas_disponibles}")
    # Filtrar por modalidad y horas
    mypes_aux = mypes[
        (mypes["modalidad_proyecto"] == modalidad) &
        (mypes["horas_requeridas"] <= horas_disponibles)
    ]
    return mypes_aux.reset_index(drop=True)

def filtrar_por_ubicacion(indice_estudiante, estudiantes, mypes, radio_km=10):
    estudiante = estudiantes.iloc[indice_estudiante]
    ubicacion_est = estudiante['ubicacion']
    
    # Obtener coordenadas del estudiante
    location_est = geolocator.geocode(ubicacion_est)
    if location_est is None:
        print("No se pudo encontrar la ubicación del estudiante.")
        return None

    coords_estudiante = (location_est.latitude, location_est.longitude)

    # Lista para guardar índices de mypes cercanas
    indices_cercanos = []

    for i, row in mypes.iterrows():
        ubicacion_mype = row['ubicacion']
        location_mype = geolocator.geocode(ubicacion_mype)

        if location_mype is None:
            continue  # Saltar si no encuentra la ubicación

        coords_mype = (location_mype.latitude, location_mype.longitude)

        # Calcular distancia
        distancia = geodesic(coords_estudiante, coords_mype).km

        if distancia <= radio_km:
            indices_cercanos.append(i)

    # Devolver solo las mypes cercanas
    return mypes.loc[indices_cercanos]

In [33]:
mypes_filtradas = filtrar_por_columnas(6, estudiantes, mypes)

Presencial 25


In [28]:
mypes_filtradas = filtrar_por_ubicacion(9,estudiantes, mypes)

KeyboardInterrupt: 

In [24]:
estudiantes.iloc[9]

Unnamed: 0                                                                9
nombre                                                           Ana Flores
correo                                           ana.flores76@correo.edu.pe
ubicacion                  Av. Universitaria, Villa María del Triunfo, Lima
skill_estrella            Análisis de laboratorio y estudios de biodiver...
motivacion_principal                                 Remuneración simbólica
hobbies                                                                Yoga
disponibilidad_semanal                                                   10
modalidad_preferida                                              Presencial
areas_interes             Aplicaciones prácticas en Biología, Tecnología...
carrera                                                            Biología
ciclo                                                                     7
Name: 9, dtype: object

# Ahora preprocesamos solo el estudiante y las mypes_filtradas

In [34]:
# Para el estudiante
estudiante = estudiantes.iloc[6]
estudiante["skill_estrella"] = preprocesar(estudiante["skill_estrella"])
estudiante["motivacion_principal"] = preprocesar(estudiante["motivacion_principal"])
estudiante["areas_interes"] = preprocesar(estudiante["areas_interes"])

/tmp/ipykernel_118986/1870823140.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estudiante["skill_estrella"] = preprocesar(estudiante["skill_estrella"])
/tmp/ipykernel_118986/1870823140.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estudiante["motivacion_principal"] = preprocesar(estudiante["motivacion_principal"])
/tmp/ipykernel_118986/1870823140.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estudiante["areas_interes"

In [35]:
# Para la mype
mypes_filtradas["area_reto"] = mypes_filtradas["area_reto"].apply(preprocesar)
mypes_filtradas["descripcion_reto"] = mypes_filtradas["descripcion_reto"].apply(preprocesar)
mypes_filtradas["habilidades_requeridas"] = mypes_filtradas["habilidades_requeridas"].apply(preprocesar)
mypes_filtradas["cultura_empresa"] = mypes_filtradas["cultura_empresa"].apply(preprocesar)

In [36]:
mypes_filtradas.head()

,Unnamed: 0,nombre_empresa,RUC,area_reto,descripcion_reto,habilidades_requeridas,cultura_empresa,horas_requeridas,modalidad_proyecto,ubicacion
0,3,Soluciones Marketing Corporación,70925802916,legal,necesitar elaborar política interno claro concisa,redacción legal derecho empresarial análisis c...,empresa joven tecnológico,20,Presencial,"Jr. Amazonas, Huaral, Lima"
1,6,Progreso Marketing SRL,34980195388,marketing,buscar desarrollar estrategia contenido captar...,seo googlar ads marketing digital,organización orientado impacto social,20,Presencial,"Av. Principal, Jaén, Cajamarca"
2,15,Progreso Exportaciones SAC,87188857461,legal,necesitar elaborar política interno claro concisa,excel normativo peruano derecho empresarial an...,startup dinámico colaborativo,25,Presencial,"Jr. San Martín, Moquegua, Moquegua"
3,17,Servicios Exportaciones Corporación,38319868934,recurso humano,mejorar proceso reclutamiento selección,psicología organizacional comunicación efectiv...,empresa familiar enfoque innovación social,25,Presencial,"Av. Principal, Chorrillos, Lima"
4,21,Comercial Diseño SPA,96105252980,diseño,rediseñar página web mejorar experiencia usuario,figma html css adobe illustrator,organización orientado impacto social,25,Presencial,"Av. Libertad, Tarapoto, San Martín"


# 2do Filtro - Creamos los vectores TF-IDF para el subset de mypes

In [ ]:
def concatenar_texto_estudiante(estudiantes):
  return (
      str(estudiantes["skill_estrella"]) + " " +
      str(estudiantes["motivacion_principal"]) + " " +
      str(estudiantes["areas_interes"])
  )

def concatenar_texto_mypes(mypes):
  return (
      str(mypes["area_reto"]) + " " +
      str(mypes["descripcion_reto"]) + " " +
      str(mypes["habilidades_requeridas"]) + " " +
      str(mypes["cultura_empresa"])
  )


def calcular_similitud_texto(estudiante, mypes_filtradas):
    # primero concatenamos todo
    texto_estudiante = concatenar_texto_estudiante(estudiante)
    textos_mypes = mypes_filtradas.apply(concatenar_texto_mypes, axis=1)

    # Ahora juntamos el texto estudiante con el texto de las mipes con lista
    corpus = [texto_estudiante] + textos_mypes.tolist()

    # Hacemos la vectorización TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)

    # Ahora hacemos el cosine similarity
    similitudes = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])[0]

    # Ahora usamos una columna de similitud
    mypes_filtradas = mypes_filtradas.copy()
    mypes_filtradas["similitud"] = similitudes

    # Ordenamos las mypes_filtradas de mayor a menor
    mypes_filtradas = mypes_filtradas.sort_values(by="similitud", ascending=False).reset_index(drop=True)

    return mypes_filtradas

In [ ]:
# 2. Filtrar MYPEs compatibles
mypes_filtradas = filtrar_por_columnas(6, estudiantes, mypes)

# 3. Calcular similitudes
mypes_con_similitud = calcular_similitud_texto(estudiante, mypes_filtradas)

# 4. Ver resultado
mypes_con_similitud[["RUC", "similitud"]]

Presencial 25


,RUC,similitud
0,53206862288,0.391569
1,14894252875,0.388043
2,33820574064,0.381881
3,81227971577,0.381151
4,45831847091,0.370377
...,...,...
660,37965841620,0.000000
661,14366201374,0.000000
662,30804139665,0.000000
663,92231914025,0.000000


In [17]:
estudiantes.iloc[6]

Unnamed: 0                                                                6
nombre                                                         Carlos López
correo                                         carlos.lópez21@correo.edu.pe
ubicacion                                           Jr. Bolognesi, Ica, Ica
skill_estrella            Diseño de material publicitario en Adobe Illus...
motivacion_principal                                               Aprender
hobbies                                              Voluntariado ambiental
disponibilidad_semanal                                                   25
modalidad_preferida                                              Presencial
areas_interes             Fundamentos de Diseño Gráfico, Metodologías bá...
carrera                                                      Diseño Gráfico
ciclo                                                                     3
Name: 6, dtype: object

In [18]:
mypes_filtradas[mypes_filtradas['RUC']==53206862288]

,Unnamed: 0,nombre_empresa,RUC,area_reto,descripcion_reto,habilidades_requeridas,cultura_empresa,horas_requeridas,modalidad_proyecto,ubicacion
239,1725,Estrategias Legal SRL,53206862288,Diseño,Requerimos material gráfico para redes y prese...,"Adobe Illustrator, HTML/CSS",Startup dinámica y colaborativa,20,Presencial,"Av. Progreso, Huánuco, Huánuco"
